In [21]:
#General use
import numpy as np
from astropy.io import fits
from astropy.cosmology import WMAP9 as cosmo

#For reading and creating tables
from astropy.table import Table, Column
import astropy.io.ascii as asc

#For photometry
from astropy import units as u
from photutils import aperture_photometry
from astropy.coordinates import SkyCoord
from photutils import SkyCircularAperture
from astropy.wcs import WCS

#supress warning untill mapping issue is resolved
import warnings
warnings.filterwarnings("ignore", message='The aperture at position')


In [16]:
#We define functions used to calculate error

def Conv_error(val,err): #uncertanty in the conversion factor from arcmin^2 to Kpc^2 using (redshift, redshift error)
    return(
            abs(((1/cosmo.kpc_comoving_per_arcmin(val+err))**2-(1/cosmo.kpc_comoving_per_arcmin(val-err))**2)/2).value
          )

def Comov_error(val,err): #Uncertanty in the comoving_distance using (redshift, redshift error)
    return(
            abs(((cosmo.comoving_distance(val+err).cgs)-(cosmo.comoving_distance(val-err).cgs))/2).value          
                )

def Flux_error(val_1,err_1): #uncertanty in flux using (counts per second, error in counts per second)
    return(
        ((val_1*flux_conv_err)**2+(flux_conv*err_1)**2)**(1/2)
            )

def Luminosity_error(flux,flux_err,dist,dist_err): #The input 'dist' is Comoving Distance
    return(
        ((4*np.pi*(dist**2)*flux_err)**2+(8*np.pi*dist*flux*dist_err)**2)**(1/2)
    )

def Sbrightness_error(lum,lum_err,conv,conv_err): #Uncertanty in surface brightness
    return(
        ((conv*lum_err/Pixel_area)**2+(lum*conv_err/Pixel_area)**2+
         (lum*conv*Pixel_area_err/(Pixel_area)**2)**2)**(1/2)
    )


In [17]:
#We define initial variables

Region_file="observed_target_info.reg"
Output_file="output NUV.csv"

flux_conv = 2.06*(10**(-16)) # A conversion factor from pixel count to flux in units of (erg cm-2 Å-1)/(N/s)
        #NUV: Flux = 2.06*10**(-16) x CPS, FUV: Flux = 1.40*10**(-15) x CPS
        #Reference: http://asd.gsfc.nasa.gov/archive/galex/FAQ/counts_background.html

flux_conv_err = 0 #error in flux_conv
    
Pixel_area=(1.5/60)**2 #Arcmin per pixel - given in arcseconds, converted to arcmin, and squared to find area in arcmin^2.

Pixel_area_err = 0 #error in Pixel_area

Pec_error = ((1+300/(299792.458))/(1-300/(299792.458)))**(1/2)-1  #The error in redshift due to peculiar velocity

In [18]:
#Create a dictionary of coordinates and redshift for each supernova enviornment

reg = Table(asc.read(Region_file, data_start=2, delimiter = "#", header_start=2)) 

cord, red = {}, {}
for row in reg: 
    cord[row[1].split(",")[0].strip('text={}')] = row[0].strip('point()').replace(',',' ')
    red[row[1].split(",")[0].strip('text={}')] = row[1].split(",")[2].strip('}').replace('z=','')
    

In [42]:
def photometry():
    #open the file and create an hdulist
    hdulist = fits.open('AIS_80_sg28-fd-int.fits')


    for x in cord:
        #Define the SN location and aperture size
        c = SkyCoord(cord[x], unit=(u.hourangle, u.deg))  
        w = WCS('AIS_80_sg28-fd-int.fits')
        w = w.all_world2pix(c.ra,c.dec, 1)
    
        if 300<w[0]<3500 or 300<w[1]<3500:

            #Find arcmin of 1kpc radius region
            r = 2*u.kpc/cosmo.kpc_comoving_per_arcmin(float(red[x]))
    
            #create an aperture
            aperture = SkyCircularAperture(c, r)

            #Perfom photometry
            phot_table = aperture_photometry(hdulist, aperture)
            if phot_table[0][0]>=0:
                name.append(x)
                redshift.append(float(red[x]))
                photom.append(phot_table[0][0])
                expt.append(hdulist[0].header['EXPTIME'])




    #close the hdulist
    hdulist.close()

In [44]:
name, redshift, photom, expt = [], [], [], []
photometry()

In [45]:
#We calculate the remaining values in the table

out = Table()
        
out['Sn Name'] = name

out['Red shift'] = redshift

out['Redshift error'] = [((1/1000)**2+(Pec_error)**2)**(1/2) for row in out] 
#Error in redshift taken as 1 in 1000 with an additional 300 km/s contribution from peculiar velocity

out['ArcMin^2 per Kpc^2 at Redshift'] =[
    (1/cosmo.kpc_comoving_per_arcmin(row[1]).value)**2 for row in out] 

out['ArcMin^2 / Kpc^2 error'] = [Conv_error(row[1],row[2]) for row in out] 

out['Photometry'] = photom

out['Exposure Time'] = expt

out['Pixel Error N^(1/2)/s'] = [(row[5]*row[6])**(1/2)/row[6] for row in out] #sqrt(cps *exposure time)/exposure time

out['Flux (erg s-1 cm-2 A-1 px-1)'] = [flux_conv*row[5] for row in out] #convert cps to flux using the conversion factor

out['Flux error (erg s-1 cm-2 A-1 px-1)'] = [Flux_error(row[5],row[7]) for row in out] 

out['Comoving Distance at Redshift (cm)'] = [cosmo.comoving_distance(row[1]).cgs.value for row in out] 

out['Comoving Error (cm)'] = [Comov_error(row[1],row[2]) for row in out]

out['Luminosity (erg s-1 A-1 px-1)'] = [row[8]*4*np.pi*row[10]**2 for row in out] #luminosity = flux*4*pi*r^2

out['Luminosity Error (erg s-1 A-1 px-1)'] = [Luminosity_error(row[8],row[9],row[10],row[11]) for row in out]

out['Surface Brightness (erg s-1 A-1 Kpc^-2)'] = [row[12]/Pixel_area*row[3] for row in out]

out['Surface Brightness error (erg s-1 A-1 Kpc^-2)'] = [Sbrightness_error(row[12],row[13],row[3],row[4]) for row in out]

out['log10 of Surface Brightness'] = [np.log10(row[14]) for row in out]

out['error in log10'] = [row[15]/(row[14]*np.log(10)) for row in out]

out.remove_columns(['Comoving Distance at Redshift (cm)', 'Comoving Error (cm)']) #remove columns we don't want in the output file

out

Sn Name,Red shift,Redshift error,ArcMin^2 per Kpc^2 at Redshift,ArcMin^2 / Kpc^2 error,Photometry,Exposure Time,Pixel Error N^(1/2)/s,Flux (erg s-1 cm-2 A-1 px-1),Flux error (erg s-1 cm-2 A-1 px-1),Luminosity (erg s-1 A-1 px-1),Luminosity Error (erg s-1 A-1 px-1),Surface Brightness (erg s-1 A-1 Kpc^-2),Surface Brightness error (erg s-1 A-1 Kpc^-2),log10 of Surface Brightness,error in log10
str352,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
ASASSN-14iu,0.016855,0.0014150577315,0.00224042863476,0.000380141103342,0.0883912713037,105.05,0.0290072569337,1.82086018886e-17,5.97549492835e-18,1.14922272689e+37,4.2331679063e+36,4.11960240806e+37,1.6707065882e+37,37.6148553033,0.176128320227
